# Soh estimation experimentation of Renault vehicles
We will try to express the soh_100_soc as:
```
soh_100_soc = charging.battery_energy / (charging.battery_level * model_battery_capacity) 
```

This expression is based on the assumption that the charging.battery_level variable is represents the actual energy in the battey and not some simple cross product.

## Setup

In [ ]:
! mkdir -p data_cache

### Imports

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from rich import print
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import plotly.express as px
from core.config import *
from scipy.stats import linregress as lr 
import numpy as np
from scipy.optimize import minimize

from transform.high_mobility.high_mobility_raw_tss import get_raw_tss
from transform.ayvens.ayvens_fleet_info import fleet_info

### Data extraction

Then we will use data find online to get the default battery capacity of each model.  
Note: *Here a model is a combinatin of the `Model` and `Type` fleet_info variables since cars of the same model with different type can have different battery capacity*.

Let's extract the raw time seriess of all the cars we have into a multi indexed tss. 

In [ ]:
raw_tss = get_raw_tss("renault")

In [ ]:
# Compter le nombre de VIN uniques
nombre_vin_uniques = raw_tss['vin'].nunique()

print(f"Le nombre de VIN différents dans tss est : {nombre_vin_uniques}")

**Note**: *There are only R135 models.*

### Time series processing
Let's implement a naive soh_100_soc estimation pipeline.  

In [ ]:
COLS_TO_CPY_FROM_FLEET_INFO = [
    "make",
    "model",
    "version",
    "dummy_soh_maker_offset",
    "dummy_soh_model_offset",
    "dummy_soh_model_slope",
    "dummy_soh_vehicle_offset",
    "capacity",
]

KEYS = [
    "slope",
    "intercept",
    "r_value",
    "p_value",
    "std_err",
]

RENAME_COLS_DICT:dict[str, str] = {
    "date_of_value": "date",
    "diagnostics.odometer": "odometer",
    "odometer.value": "odometer",
    "diagnostics.odometer": "odometer",
    "mileage_km": "odometer",
    "mileage": "odometer",
    "charging.battery_energy": "battery_energy",
    "charging.estimated_range": "estimated_range",
    "charging.battery_level": "soc",
    "soc_hv_header": "soc",
    "charging.battery_energy": "battery_energy",
    "charging.battery_level": "battery_level",
}

COLS_TO_KEEP = [
    "date",
    "soc",
    "odometer",
    "estimated_range",
    "battery_energy",
    "soc",
    "vin",
]

COL_DTYPES = {
    "soc": "float",
    "odometer": "float",
    "estimated_range": "float",
    "battery_energy": "float",
    "soc": "float",
    "vin": "string",
    "capacity": "float",
}

In [ ]:
tss:DF = (
    raw_tss
    .merge(fleet_info[COLS_TO_CPY_FROM_FLEET_INFO], on="vin", how="left")
    .assign(capacity=52)# Hot fix, some capacity values of the fleet info are incorrectly equal R135 (the version of the car) instead of 52 (the capacity of the car)  
    .rename(columns=RENAME_COLS_DICT)
    .eval("soc = battery_level * 100")
    .astype(COL_DTYPES, errors="ignore")
    .eval("expected_battery_energy = capacity * battery_level")
    .eval("simple_soh = 100 * battery_energy / expected_battery_energy") 
)
tss.columns

## EDA

## Assumption verification
First, we will verify that the `soc` and `battery_energy` are two "real" variables.  
That is, none of them is calculated from the other.

In [ ]:
# Compter le nombre de VIN uniques
nombre_vin_uniques = tss['vin'].nunique()

print(f"Le nombre de VIN différents dans tss est : {nombre_vin_uniques}")

In [ ]:
px.scatter(tss, x="soc", y="battery_energy", color="vin")

In [ ]:
px.box(tss, x="soc", y="battery_energy")

Looking at this scatter plot we can see that:
- The two variables are in fact two real variables instead of one being a synthetic variable calculated from the other.  
- The difference is much more important at high `soc` values.

We can also see that the battery_energy at soc 1% is not actually 1% of the total energy.  
This implies that we can't simply express the energy at a certain soc that a 100% soh_100_soc battery would have as capcity * soc / 100.  

We will therfore use the parameters of a linear regression trained on soc -> battery_energy to compute the soh_100_soc.   
We will then be able to express the soh_100_soc either as:
- the predicted 100% soc battery_energy / capacity.
- the predicted integral of battery_energy from 1% to 100% soc divided by the default integral of capacity.

## battery_energy and soc points extraction

In [ ]:
energy_over_soc = (
    tss
    .groupby(["vin", "soc"])
    .agg({"soc":"median", "battery_energy":"median", "version": "first"})
    .reset_index(level=0)
)
energy_over_soc

## linear regression models training

We will constrain the intercepts value into an aribitrary range to increase the quality of the fitting.  
We could add more constrains to further improve the fitting.  

In [ ]:
def lr_with_constrained_intercept(tss) -> Series:
    # Get X and y values from the dataframe
    X = tss["soc"].values
    y = tss["battery_energy"].values
    
    # Define the objective function to minimize sum of squared residuals
    def objective_function(params, X, y):
        slope, intercept = params
        y_pred = slope * X + intercept
        return np.sum((y - y_pred) ** 2)
    
    # Define the constraint: intercept between 0.5 and 2.5
    def constraint_intercept(params):
        intercept = params[1]
        return intercept - 0.5, 2.5 - intercept
    
    # Add both constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)[0]},  # intercept >= 0.5
        {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)[1]},  # intercept <= 2.5
    ]
    
    # Initial guess for slope and intercept
    initial_guess = np.array([0, 1])

    # Minimize the objective function with the constraints
    result = minimize(objective_function, initial_guess, args=(X.squeeze(), y), constraints=constraints)

    # Extract the optimal slope and intercept
    slope, intercept = result.x

    return Series({"slope": slope, "intercept": intercept})


In [ ]:
soc_to_energy_relation_descr_stats:DF = (
    tss
    .dropna(subset=["soc", "battery_energy"], how="any")
    .groupby("vin")
    .apply(
        lr_with_constrained_intercept,
        include_groups=False,
    )
    .eval("soc_0_battery_energy = intercept")
    .eval("soc_100_battery_energy = intercept + slope * 100")
    .eval("integral = (slope * (100**2) / 2) + (intercept * 100)")
    .eval("soh_100_soc = soc_100_battery_energy / 51 * 100")
    .merge(tss.groupby("vin").agg({"odometer": "max"}), on="vin", how="left")
    .merge(fleet_info[COLS_TO_CPY_FROM_FLEET_INFO], on="vin", how="left")
)
soc_to_energy_relation_descr_stats

Let's plot the linear regressions.  

In [ ]:
plt_soc_to_energy_relation_descr_stats = (
    soc_to_energy_relation_descr_stats
    .loc[:, ["soc_0_battery_energy", "soc_100_battery_energy"]]
    .T
    .unstack()
    .reset_index()
    .rename(columns={"level_1": "soc", 0: "battery_energy"})
    .replace({"soc_0_battery_energy": 0, "soc_100_battery_energy": 100})
)

px.line(
    plt_soc_to_energy_relation_descr_stats,
    x="soc",
    y="battery_energy",
    color="vin"
)

In [ ]:
soc_to_energy_relation_descr_stats.sort_values("soh_100_soc")

Let's see how coherent our estimation over odometer looks like.  

In [ ]:
px.scatter(
    soc_to_energy_relation_descr_stats,
    x="odometer",
    y="soh_100_soc",
    trendline="ols",
    color="version"
)

### integral soh

Let's compute the soh as the `battery_energy` integral divided by the default `battery_energy` integral.  

In [ ]:
DFLT_INTEGRAL = 51/100 * 100**2 / 2
soc_to_energy_relation_descr_stats["soh_integral"] = soc_to_energy_relation_descr_stats.eval(f"integral / {DFLT_INTEGRAL} * 100")

px.scatter(
    soc_to_energy_relation_descr_stats,
    x="odometer",
    y="soh_integral",
    color="version",
    trendline="ols",
)

In [ ]:
soc_to_energy_relation_descr_stats = (
    soc_to_energy_relation_descr_stats
    .assign(simple_soh=tss["simple_soh"].mask(lambda soh_100_soc: soh_100_soc < 50, np.nan).groupby(tss["vin"]).mean())
)

In [ ]:
px.scatter(
    soc_to_energy_relation_descr_stats,
    x="odometer",
    y="simple_soh",
    color="version",
    trendline="ols",
)

### Sohs comparaisons

In [ ]:
descriptions = soc_to_energy_relation_descr_stats[["soh_100_soc", "soh_integral", "simple_soh"]].describe()

descriptions

In [ ]:
corr_with_odo = soc_to_energy_relation_descr_stats.corr(numeric_only=True).loc[["odometer"], ['soh_100_soc', 'soh_integral', 'simple_soh']]

corr_with_odo

# Conclusion
There does not seem to be a significant improvement using the linear regression (-_-)...